1. 문서 내용을 읽는다
2. 문서를 쪼갠다.
    - 토큰수 초과로 답변을 생성하지 못할 수 있고
    - 문서가 길면 답변 생성이 오래걸림
3. 임베딩 -> 벡터 데이터베이스 저장
4. 질문이 있을 때, 벡터 데이터베이스에 유사도 검색
5. 유사도 검색으로 가져온 문서를 LLM에 질문과 같이 전달

In [9]:
from dotenv import load_dotenv
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from tenacity import wait_chain

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500, # 청크 하나
    chunk_overlap=200, # 자를때 딱딱 나눠서 자르는게 아니라 어느정도 겹치게 잘라주는 것
)

loader = Docx2txtLoader('./tax.docx')
document_list = loader.load_and_split(text_splitter=text_splitter)

In [13]:
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings

load_dotenv()
embedding = OpenAIEmbeddings(model='text-embedding-3-large') # 디폴트 모델이 별로라 성능 좋은 모델 넣어준

In [45]:
from langchain_chroma import Chroma

# database = Chroma.from_documents(documents=document_list, embedding=embedding, collection_name='chroma-tax',persist_directory="./chroma")

database = Chroma(collection_name='chroma-tax', persist_directory='./chroma', embedding_function=embedding)

In [21]:
query = '연봉 5천만원의 직장인의 소득세는 얼마인가요?'
retrieved_docs = database.similarity_search(query, k=3)

In [34]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model='gpt-4o')

In [24]:
# prompt = f"""[Identity]
# - 당신은 최고의 한국 소득세 전문가 입니다
# - [Context]를 참고해서 사용자의 질문에 답변해주세요
#
# [Context]
# {retrieved_docs}
#
# Question: {query}
# """

In [25]:
# ai_message = llm.invoke(prompt)

In [26]:
# ai_message.content

'연봉 5천만원의 직장인의 소득세 계산은 다음과 같은 단계로 이루어집니다. 한국의 소득세는 기본세율을 사용하며, 각 구간에 따라 세율이 다르게 적용됩니다. 2023년 기준으로 설명드리겠습니다. \n\n1. **과세표준**을 계산하기 위해 우선 연봉에서 근로소득공제 및 각종 소득 공제를 차감해야 합니다. \n\n2. **근로소득공제**: 연봉 5천만원에 대한 근로소득공제는 다음과 같습니다.\n   - 3,000만원 이하: 소득의 70%\n   - 3,000만원 초과 ~ 4,500만원 이하: 2,100만원 + 3,000만원 초과분의 40%\n   - 4,500만원 초과 ~ 1억원 이하: 2,700만원 + 4,500만원 초과분의 15%\n\n   연봉이 5천만원이므로, 근로소득공제는 2,700만원 + (5,000만원 - 4,500만원) * 15% = 2,775만원이 됩니다.\n\n3. 근로소득공제를 연봉에서 차감하여 **과세표준**을 계산합니다:\n   - 과세표준 = 5,000만원 - 2,775만원 = 2,225만원\n\n4. **소득세율 적용**:\n   - 1,200만원 이하: 6%\n   - 1,200만원 초과 ~ 4,600만원 이하: 72만원 + (1,200만원 초과분의 15%)\n   \n   과세표준 2,225만원이므로, 1,200만원 초과분에 대해 15% 세율이 적용됩니다.\n   - 소득세 = 72만원 + (2,225만원 - 1,200만원) * 15% = 72만원 + 1,025만원 * 0.15 = 72만원 + 153.75만원 = 225.75만원\n\n따라서 연봉 5천만원인 직장인의 소득세는 대략 225만 7,500원이 됩니다.\n\n이 계산은 단순화를 위해 다른 각종 세액 감면, 특별 공제 등을 적용하지 않았습니다. 실제 세부 항목을 적용할 경우 결과가 달라질 수 있습니다. 정확한 정보는 국세청의 연말정산 프로그램을 활용하시기 바랍니다.'

In [28]:
%pip install -U langchain langchainhub --quiet

Note: you may need to restart the kernel to use updated packages.


In [30]:
from langchain import hub

prompt = hub.pull("rlm/rag-prompt")


/opt/anaconda3/envs/langchain-env/lib/python3.10/site-packages/langsmith/client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})])

In [50]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=database.as_retriever(),
    chain_type_kwargs={"prompt": prompt}
)

In [51]:
ai_message = qa_chain({"query": query})

In [52]:
ai_message

{'query': '연봉 5천만원의 직장인의 소득세는 얼마인가요?',
 'result': '연봉 5천만 원의 직장인의 소득세는 기본세율에 따라 계산됩니다. 기본세율은 소득 구간에 따라 누진세율로 적용되며, 연봉과 각종 공제 항목에 따라 정확한 세액이 결정됩니다. 구체적인 계산을 위해서는 소득공제와 세액공제를 감안하여 세무 대리인과 상담하거나 국세청 홈택스를 이용하세요.'}